# 元类

## metaclass的超越变形特性

YAML是一个用于序列化和反序列化的工具，其`YAMLObject`的一个超越变形能力是它的子类支持序列化和反序列化操作。

In [42]:
import yaml
class Monster(yaml.YAMLObject):
    yaml_loader = yaml.SafeLoader
    yaml_tag=u'!Monster'
    def __init__(self,name,hp,ac,attacks):
        self.name=name
        self.hp=hp
        self.ac=ac
        self.attacks=attacks
    def __repr__(self):
        return "{}(name={},hp={},ac={},attacks={})".format(
            self.__class__.__name__,self.name,self.hp,self.ac,self.attacks)

In [43]:
print(yaml.dump(Monster('martin',[3,6],ac=9,attacks=['BITE','HURT'])))

!Monster
ac: 9
attacks:
- BITE
- HURT
hp:
- 3
- 6
name: martin



In [45]:
info='''
!Monster
ac: 9
attacks:
- BITE
- HURT
hp:
- 3
- 6
name: martin
'''
obj=yaml.safe_load(info)

In [46]:
obj.ac

9

使用yaml.load、dump能在不知道任何类型的前提下将一个YAMLObject的子类序列化和反序列化，这让超动态配置编程(通过配置文件控制加载的类)成了可能。

## YAML的序列化和反序列化如何实现的？

实现这个功能需要做到两步：
- 实现注册函数，让YAML知道tag对应的Python类。
- 定义类时手动调用注册函数。
实际上YAML是如何实现的呢？

```python
# YAMLObject通过metaclass来注入注册函数调用，避免手动调用
class YAMLObject(metaclass=YAMLObjectMetaclass):
    """
    An object that can dump itself to a YAML stream
    and load itself from a YAML stream.
    """
    ...

class YAMLObjectMetaclass(type):
    """
    The metaclass for YAMLObject.
    """
    def __init__(cls, name, bases, kwds):
        super(YAMLObjectMetaclass, cls).__init__(name, bases, kwds)
        if 'yaml_tag' in kwds and kwds['yaml_tag'] is not None:
            # 注册函数
            cls.yaml_loader.add_constructor(cls.yaml_tag, cls.from_yaml)
            cls.yaml_dumper.add_representer(cls, cls.to_yaml) 
```

## metaclass的工作原理？

### Python中自定义类都是type的实例

In [47]:
class MyClass:
    pass
type(MyClass)

type

### 自定义类是type类`__call__`运算符的重装

In [49]:
class MyClass:
    data=1

obj=MyClass()
print(MyClass,obj)
print(obj.data)

<class '__main__.MyClass'> <__main__.MyClass object at 0x000000000BF26080>
1


In [52]:
my_class=type('MyClass',(),{'data':1})
obj=my_class()
print(obj.data)
print(my_class,obj)

1
<class '__main__.MyClass'> <__main__.MyClass object at 0x000000000BF66A20>


从上面的实验可以看出,创建一个类对象：`class=type(classname,superclass,attributedict)`,这是对type的`__call__`的重载，会进一步调用`type.__new__(classname,superclass,attributedict)`和`type.__init__(classname,superclass,attributedict)`

### metaclass是type的子类，通过替换type的`__call__`运算符重载机制，超越变形正常的类。

当类MyClass的metaclass被设置为其他的MyMeta后，MyClass将调用MyMeta的`__call__`运算符重载，而不是type。

## metaclass的风险

metaclass会扭曲正常的Python类型模型

## 实验

In [58]:
class MyMeta(type):
    def __init__(cls, name, bases, kwds):
        print('MyMeta init', cls.__name__)
        super(MyMeta, cls).__init__(name, bases, kwds)
        print('MyMeta init')
        print(name)
        print(bases)
        print(kwds)

    def __new__(cls, *args, **kwargs):
        print('MyMeta new')
        print('MyMeta new', cls.__name__)
        return type.__new__(cls, *args, **kwargs)

    def __call__(cls, *args, **kwargs):
        print('MyMeta call')
        print('MyMeta call', cls.__name__)
        obj = cls.__new__(cls, *args, **kwargs)
        cls.__init__(cls, *args, **kwargs)
        return obj


class Foo(metaclass=MyMeta):
    def __init__(self, name):
        print('foo init')
        self.name = name

    def __new__(cls, *args, **kwargs):
        print('foo new')
        return object.__new__(cls)


f = Foo('martin')

MyMeta new
MyMeta new MyMeta
MyMeta init Foo
MyMeta init
Foo
()
{'__module__': '__main__', '__qualname__': 'Foo', '__init__': <function Foo.__init__ at 0x000000000BAFBB70>, '__new__': <function Foo.__new__ at 0x000000000BAFB158>}
MyMeta call
MyMeta call Foo
foo new
foo init


解析：
- Foo(metaclass=MyMeta)：通过MyMeta创建一个新的类对象，所有会调用MyMeta的`__new__`和`__init__`方法。
-  Foo('martin')：创建一个对象，会调用MyMeta的`__call__`方法，`__call__`方法中调用Foo的`__new__`和`__init__`方法。